In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
### PANEL GŁÓWNY Numpy ### - Zmienne niezbędne dla wykonania zbioru danych przez Numpy biorą się stąd. 
# Szczegóły słabiej opisanych - zeszyt kursowy 1

observations = 1000  # Ustaw ilość obserwacji (Startowa: 1000)
vnum = 1             # Liczba zmiennych
noic = 10            # Szum w bazie przypadków treningowych.
noit = 1             # Szum wygenerowany dla funkcji testującej (docelowej).
model = [13,7,-12]   # Parametry modelu losowego dla funkcji a*xs + b*xz + c + szum.

#### 1. Tworzenie i składowanie danych

In [3]:
xs = np.random.uniform(low = -noic, high = noic, size =(observations,vnum)) # Gen. zakr. los. zm. wej.(patrz zeszyt 1)
zs = np.random.uniform(-noic,noic,(observations,vnum)) # J.w.- bez opisów bo zbędne. Poz. dla orientacji.

generated_inputs = np.column_stack((xs,zs)) # Składanie z pow. matrycy w formacie observationsx2. (Patrz zeszyt 1)

noise = np.random.uniform(-noit,noit,(observations,1)) # Gen. zakr. los. (Patrz zeszyt 1)

generated_targets = model[0]*xs + model[1]*zs + model[2] + noise # Funkcja symulująca zadane cele. Do niej dąży model. (Patrz zeszyt 1)

np.savez('My_Nums', inputs = generated_inputs, targets = generated_targets) # Tworzy plik My_Nums.npz w tym samym miejscu co ten zeszyt. 
# Zawiera podane matryce w formie ndarray, co dla obsługi TF jest bardzo wygodne.

### 2. Tutaj zaczyna się TensorFlow
Uwaga odnośnie używania TF - obiekty TF nie wykonują się same.
Trzeba im wydać konkretne polecenie. 

Jeśli chcesz na to spojrzeć z innej strony, 
to zeszyt badawczy 2a zawiera opis i przykłady poszczególnych elementów TF.

In [4]:
### Panel Główny TF ### - tutaj będę wyciągał zmienne dla modelu TF,
# żeby było jasne co jest do czego. Normalnie wszystkie zmienne,
# którymi miałbym potencjalnie poźniej manipulować zbieram dla porządku 1 miejscu.

input_size = 2 # Ilość zmiennych wejściowych - tutaj mamy dwie (x,z)
output_size = 1 # Nasza funkcja ma jedno wyjście - y.

#### 3. Rysujemy model

In [5]:
# Tworzy podstawnik do którego będziemy podstawiać dane wejściowe z naszego zbioru danych w pliku npz.
# 'float32' to miara precyzji (32bit) - standard dla Pythona to float64, ale tyle na ogół wystarczy.
# W nawiasie kwadratowym znajdują się wymiary podstawianych matryc. 'None' jest w pierwszym nie dlatego,
# że nasze dane nie mają pierwszego wymiaru (tj. ilość rekordów), ale że nie musimy tego określać.
# To przydatne, bo nie trzeba dzięki temu znać dokładnej ilości obserwacji ani pisać odpowiedniego kodu samemu.
# Ważne jedynie, żeby podać ilość zmiennych wejściowych (input_size) i wyjściowych (output_size).
inputs = tf.placeholder(tf.float32,[None,input_size])
# Podstawnik gdzie podamy funkcji dane testowe (cele) - jest w tym samym kształcie co dane wyjściowe,
# więc korzystamy ze zmiennej opisującej ilość zmiennych wyjściowych (output_size).
targets = tf.placeholder(tf.float32,[None,output_size])

### To były podstawniki. Zauważ, że pomiędzy iteracjami nie zachowują one swojej wartości. Są one dla TF
### tymczasową zmienną zewnętrzną z której czerpie dane. Do dalszego działania będziemy potrzebować zmiennych.
### Zwróć uwagę gdzie w TensorFlow są wielkie litery jak na przykład w tf.Variable. To Ci oszczędzi stresów.

# Tworzymy zmienne gdzie model będzie zwracał Wagi (weights) i Obciążenia (biases). Metoda tf.random działa
# w sposób podobny do tego co stworzyliśmy na poprzednim arkuszu. Szczegółowo o niej później.
# Kształty zmiennych są rzecz jasne takie same jak w zeszycie 1, bo wynikają z tej samej logiki.
weights = tf.Variable(tf.random_uniform([input_size,output_size], minval=-0.1, maxval = 0.1))
biases = tf.Variable(tf.random_uniform([output_size], minval=-0.1, maxval = 0.1))

# Tworzymy zmienne wyjściowe. Metoda tf.matmul działa na podobnej zasadzie do użytego w zeszycie 1
# Iloczynu Skalarnego (metoda np.dot - ang. "dot product"), ale jest zgeneralizowana do tensorów.
# Tak samo w nawiasie wstawiamy tam to co chcemy przez siebie przemnożyć.
outputs = tf.matmul(inputs,weights) + biases

#### 4. Deklaracja Funkcji Celu*, oraz metody optymalizacji


*in. f. straty [podobnej 'L2' użyliśmy w zeszycie 1] - ang. Objective Function

Więcej w temacie Funkcji Celu pod: https://pl.wikipedia.org/wiki/Funkcja_celowa

In [6]:
# Deklarujemy strukturę funkcji straty dla normy L2 tak jak w Zeszycie 1.
# W TF jest biblioteka takich funkcji 'tf.losses' z której wybieramy
# interesującą nas norma L2 (euklidesowa**) jest tam pod hasłem 'mean_squared_error'.
# Atrybutem pierwszym jest 'labels' reprezentujące klucz prawidłowych odpowiedzi dla uczenia się
# funkcji. Np "1" jeśli na danym "obrazku" widnieje "1" a funkcja ma kilka odpowiedzi do wyboru.
# Drugi atrybut to 'predictions' - logicznie są to dane wyjściowe, czyli wyniki zgadywania naszego modelu.
# Kropka po cyfrze 2 jest tam celowo, żeby na pewno otrzymać w wyniku liczbę typu float.
mean_loss = tf.losses.mean_squared_error(labels=targets, predictions=outputs) / 2. 

** O normach, w tym Euklidesowej: https://pl.wikipedia.org/wiki/Przestrze%C5%84_unormowana

In [7]:
# Deklarujemy metodę optymalizacji (treningu) funkcji. Z zasobów metod 'tf.train' wybieramy tę co ostatnio
# czyli metodę gradientu prostego** (GradientDescentOptimizer), oraz ustalamy tempo uczenia się modelu (learning_rate)
# Do podfunkcji '.minimize' podstawiamy ustaloną wyżej funkcję straty (mean_loss).
optimize = tf.train.GradientDescentOptimizer(learning_rate=0.05).minimize(mean_loss)

** O Metodzie gradientu prostego: https://pl.wikipedia.org/wiki/Metoda_gradientu_prostego

#### 5. Przygotowanie do wykonywania zmiennych

In [12]:
# Kanonicznie 'sess' jest zmienną, którą nazywa się sesję TF. Tak jak w Klasach używa się self. Dla klarowności.
# Obiekty biblioteki tf nie podlegają automatycznemu wykonywaniu wynikającemu z kolejności jak to bywa w Pythonie.
# Jest to użyteczne ze względu na specyfikę uczenia maszynowego, o czym przekonasz się wkrótce.
# Wywołanie sesji jest jednoznaczne z wykonaniem zadań TF. Raz wywołana, sesja jest aktywna do czasu zamknięcia
# Bądź też zamyka się automatycznie, jeśli użyjemy jej w odpowiedniej strukturze (Zeszyt badań 2a)
sess = tf.InteractiveSession()

#### 6. Inicjalizacja zmiennych

In [19]:
# To jest ta klasyczna część właściwie każdego skryptu TF - Inicjalizator zmiennych
# W poprzednich wersjach TF nazywał się inaczej, jednak działa tak samo. (por. Zeszyt badań 2)
# W tym momencie przypisujemy go do zmiennej 'initializer' żeby łatwiej później go wywołać w sesji.
initializer = tf.global_variables_initializer()

# W tym miejscu wywołujemy zdefiniowane wcześniej, zainicjowane zmienne.
sess.run(initializer)

#### 7. Wczytywanie danych treningowych

In [21]:
# Przypisujemy wygenerowane wcześniej w NumPy dane w formacie .npz do zmiennej 'training_data'
# za pomocą np.load - zauważ, że to co NumPy kodował, teraz rozkodowuje.
# Ta metoda ładuje plik z tego samego miejsca w którym znajduje się skrypt,
# chyba, że podasz inną ścieżkę.
training_data = np.load('My_Nums.npz')

#### 8. Uczenie modelu

In [22]:
# Ustalamy ilość iteracji. Kanonicznie jest to 'e', dla 'epoki' (ang. 'epoch')
# Dla maszyny każda kolejna iteracja w której się uczy to właśnie 'epoka'.
for e in range(100):
    # Pętla zwraca wartość dla operacji 'optimize', oraz 'mean_loss' - Patrz. pkt.4 "Deklaracja Funkcji Celu",
    # dla każdej 'epoki' tzn. 'None' dla 'optimize', do czego stosujemy '_', żeby zignorować tę wartość,
    # oraz 'curr_loss' (ang. 'current loss' - aktualna strata) dla funkcji straty 'mean_loss'.
    _, curr_loss = sess.run([optimize, mean_loss],
                            # Logika sess.run jest taka, że w pierwszym parametrze podstawiamy listę [w, nawiasach_kwadratowych]
                            # tych rzeczy, które chcemy, aby zostały wykonane,
                           feed_dict = {inputs: training_data['inputs'], targets: training_data['targets']})
                            # zaś drugi parametr, feed_dict to słownik z którego czerpie algorytm.
                            # Składnia: feed_dict = {podstawnik1 : dane, podstawnik2 : dane},
                            # Logika działa tu w naszym wypadku następująco: "Weż dane opisane jako 'inputs' ze zmiennej training_data
                            # i przypisz je do podstawnika 'inputs'. Potem weź dane opisane jako 'targets' ze zmiennej training_data
                            # i przypisz je do podstawnika 'targets'. Zmienna training_data to jak pamiętamy nasze dane NumPy (pkt. 7).
  
    print(curr_loss) #Drukuje listę funkcji celu dla wszystkich epok po kolei, dzięki czemu widzimy jak model się "uczy".    

3724.3389
1822.4303
909.4385
467.01273
250.24538
142.51315
87.8728
59.30484
43.674843
34.557796
28.789919
24.801207
21.804653
19.39995
17.378742
15.628785
14.086336
12.712594
11.48182
10.375473
9.379104
8.480856
7.670585
6.939448
6.279587
5.6840086
5.146416
4.6611476
4.223101
3.8276823
3.470737
3.14852
2.857651
2.5950842
2.3580673
2.1441102
1.950967
1.7766192
1.6192317
1.4771603
1.348909
1.2331411
1.1286324
1.0342915
0.94913244
0.87225735
0.8028611
0.74021804
0.6836697
0.63262355
0.5865435
0.54494524
0.5073963
0.47350007
0.44290167
0.4152799
0.3903475
0.36783895
0.34752038
0.32918036
0.3126244
0.2976779
0.28418675
0.27200654
0.26101282
0.25108922
0.24212964
0.23404309
0.22674277
0.22015317
0.2142039
0.20883414
0.20398614
0.1996106
0.19565989
0.19209424
0.18887536
0.18596958
0.18334681
0.18097922
0.17884193
0.17691232
0.17517087
0.17359862
0.17217901
0.17089795
0.16974144
0.16869746
0.16775501
0.1669042
0.16613598
0.16544299
0.16481702
0.16425243
0.16374236
0.16328199
0.16286658
0.16249

In [23]:
sess.close()